<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/population_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人口マスタ

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [62]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse
import math

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [63]:
def get_file_from_url(url, save_path):
    res = requests.get(url)
    if res.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(res.content)
    else:
        print(res)
        raise Exception(f'status_code: {res.status_code} ({url})')

### 全区
- [平成27年国勢調査 東京都区市町村町丁別報告](https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd-15index.htm)

In [64]:
temp_dir = '/content/tmp'
os.makedirs(temp_dir, exist_ok=True)

In [65]:
# 人口・面積
zip_path_1 = os.path.join(temp_dir, '001.zip')
zip_url_1 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv01100000.zip'
# get_file_from_url(zip_url_1, zip_path_1)

# 世帯数
zip_path_2 = os.path.join(temp_dir, '002.zip')
zip_url_2 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv02100000.zip'
# get_file_from_url(zip_url_2, zip_path_2)

# 労働力人口・従業上の地位
zip_path_5 = os.path.join(temp_dir, '005.zip')
zip_url_5 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv05100000.zip'
# get_file_from_url(zip_url_5, zip_path_5)

# 住居の種類
zip_path_6 = os.path.join(temp_dir, '006.zip')
zip_url_6 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv06100000.zip'
# get_file_from_url(zip_url_6, zip_path_6)

# 住宅の建て方
zip_path_7 = os.path.join(temp_dir, '007.zip')
zip_url_7 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv07100000.zip'
# get_file_from_url(zip_url_7, zip_path_7)

In [66]:
!unzip -o -d $temp_dir $zip_path_1
!unzip -o -d $temp_dir $zip_path_2
!unzip -o -d $temp_dir $zip_path_5
!unzip -o -d $temp_dir $zip_path_6
!unzip -o -d $temp_dir $zip_path_7

Archive:  /content/tmp/001.zip
  inflating: /content/tmp/kd15zv01100000.csv  
Archive:  /content/tmp/002.zip
  inflating: /content/tmp/kd15zv02100000.csv  
Archive:  /content/tmp/005.zip
  inflating: /content/tmp/kd15zv05100000.csv  
Archive:  /content/tmp/006.zip
  inflating: /content/tmp/kd15zv06100000.csv  
Archive:  /content/tmp/007.zip
  inflating: /content/tmp/kd15zv07100000.csv  


## 人口・面積

In [67]:
csv_file_1 = 'kd15zv01100000.csv'
df_all_popdata = pd.read_csv(os.path.join(temp_dir, csv_file_1), dtype='str')
df_all_popdata = df_all_popdata[df_all_popdata['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_popdata.shape)

(19818, 160)


In [68]:
df_popdata = df_all_popdata[[
    '地域ＩＤ',
    '地域階層フラグ／丁目･小字相当',
    '表側表章地域（階層別）／区市町村階層以上の地域名称',
    '表側表章地域（階層別）／町・大字相当階層の地域名称',
    '表側表章地域（階層別）／丁目・小字等階層の地域名称',
    '表側／男女の別',
    '人口総数（人）',
    '面積（平方キロメートル） ',
    '人口密度（人／平方キロメートル） ',
    '年齢（５歳階級）／0～4歳（人）',
    '年齢（５歳階級）／5～9歳（人）',
    '年齢（５歳階級）／10～14歳（人）',
    '年齢（５歳階級）／15～19歳（人）',
    '年齢（５歳階級）／20～24歳（人）',
    '年齢（５歳階級）／25～29歳（人）',
    '年齢（５歳階級）／30～34歳（人）',
    '年齢（５歳階級）／35～39歳（人）',
    '年齢（５歳階級）／40～44歳（人）',
    '年齢（５歳階級）／45～49歳（人）',
    '年齢（５歳階級）／50～54歳（人）',
    '年齢（５歳階級）／55～59歳（人）',
    '年齢（５歳階級）／60～64歳（人）',
    '年齢（５歳階級）／65～69歳（人）',
    '年齢（５歳階級）／70～74歳（人）',
    '年齢（５歳階級）／75～79歳（人）',
    '年齢（５歳階級）／80～84歳（人）',
    '年齢（５歳階級）／85～89歳（人）',
    '年齢（５歳階級）／90～94歳（人）',
    '年齢（５歳階級）／95～99歳（人）',
    '年齢（５歳階級）／100歳以上（人）',
    '年齢（５歳階級）／年齢不詳（人）',
    '平均年齢（歳）',
    '（再掲）外国人（人） ',
]]
df_popdata.columns = [
    'town_id',
    '地域階層フラグ',
    '市区町村名',
    '町名',
    '町丁目',
    '男女',
    '人口総数（人）',
    '面積（km2）',
    '人口密度(人/km2)',
    '0〜4歳（人）',
    '5〜9歳（人）',
    '10〜14歳（人）',
    '15〜19歳（人）',
    '20〜24歳（人）',
    '25〜29歳（人）',
    '30〜34歳（人）',
    '35〜39歳（人）',
    '40〜44歳（人）',
    '45〜49歳（人）',
    '50〜54歳（人）',
    '55〜59歳（人）',
    '60〜64歳（人）',
    '65〜69歳（人）',
    '70〜74歳（人）',
    '75〜79歳（人）',
    '80〜84歳（人）',
    '85〜89歳（人）',
    '90〜94歳（人）',
    '95〜99歳（人）',
    '100歳以上（人）',
    '年齢不詳（人）',
    '平均年齢（歳）',
    '外国人（人）'
]
df_popdata = df_popdata.replace('X', 0).replace('-', 0)
df_popdata = df_popdata[
    ((df_popdata['地域階層フラグ'] == '●')|(df_popdata['地域階層フラグ'] == '○')) &
    (df_popdata['市区町村名'].str.endswith('区')) &
    (df_popdata['男女'] == '総数')
].drop(columns=['地域階層フラグ', '男女']).reset_index(drop=True)

df_popdata.loc[df_popdata['町丁目'].isna(), '町丁目'] = df_popdata['町名']
print(df_popdata.shape)

(3145, 31)


In [69]:
pop_cols = [col for col in df_popdata.columns if '（人）' in col]
for col in pop_cols:
    df_popdata[col] = df_popdata[col].astype(int)

df_popdata['面積（km2）'] = df_popdata['面積（km2）'].astype(float)
df_popdata['人口密度(人/km2)'] = df_popdata['人口密度(人/km2)'].astype(float)
df_popdata['平均年齢（歳）'] = df_popdata['平均年齢（歳）'].astype(float)

In [70]:
df_popdata.loc[
    (df_popdata['人口密度(人/km2)'] == 0) &
    (df_popdata['面積（km2）'] > 0),
    '人口密度(人/km2)'
] = df_popdata['人口総数（人）'] / df_popdata['面積（km2）']
df_popdata['人口密度(人/km2)'] = df_popdata['人口密度(人/km2)'].round(1)

In [71]:
# 人口、人口密度が低い町は対象外にする
df_popdata['対象'] = 1
df_popdata.loc[
    ((df_popdata['人口総数（人）'] < 30) |
    (df_popdata['人口密度(人/km2)'] < 500)),
    '対象'
] = 0
print(len(df_popdata[df_popdata['対象'] == 0]))

128


## 世帯数

In [72]:
csv_file_2 = 'kd15zv02100000.csv'
df_all_family = pd.read_csv(os.path.join(temp_dir, csv_file_2), dtype='str')
df_all_family = df_all_family[df_all_family['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_family.shape)

(6606, 41)


In [73]:
df_family = df_all_family[[
    '地域ＩＤ',
    '世帯総数／一般世帯数／総数（世帯）',
    '世帯総数／一般世帯数／世帯人員が1人（世帯）',
    '世帯総数／一般世帯数／世帯人員が2人（世帯）',
    '世帯総数／一般世帯数／世帯人員が3人（世帯）',
    '世帯総数／一般世帯数／世帯人員が4人（世帯）',
    '世帯総数／一般世帯数／世帯人員が5人（世帯）',
    '世帯総数／一般世帯数／世帯人員が6人（世帯）',
    '世帯総数／一般世帯数／世帯人員が7人以上（世帯）',
    '世帯総数／一般世帯人員（人）',
    '世帯総数／一般世帯の1世帯当たり人員（人）',
]].copy()
df_family.columns = [
    'town_id',
    '世帯総数',
    '1人世帯',
    '2人世帯',
    '3人世帯',
    '4人世帯',
    '5人世帯',
    '6人世帯',
    '7人以上世帯',
    '世帯人員（人）',
    '1世帯当たり人員（人）',
]
df_family = df_family.replace('X', 0).replace('-', 0)
for col in df_family.columns:
    if col == 'town_id':
        continue
    elif col == '1世帯当たり人員（人）':
        df_family[col] = df_family[col].astype(float)
    else:
        df_family[col] = df_family[col].astype(int)

df_family['5人以上世帯'] = df_family['5人世帯'] + df_family['6人世帯'] + df_family['7人以上世帯']
df_family = df_family[[
    'town_id', '世帯総数', '1人世帯', '2人世帯', '3人世帯', '4人世帯', '5人以上世帯', '世帯人員（人）', '1世帯当たり人員（人）',
]].reset_index(drop=True)
print(df_family.shape)
assert df_family['town_id'].nunique() == len(df_family)

(6606, 9)


## 労働力人口・従業上の地位

In [74]:
csv_file_5 = 'kd15zv05100000.csv'
df_all_labor = pd.read_csv(os.path.join(temp_dir, csv_file_5), dtype='str')
df_all_labor = df_all_labor[df_all_labor['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_labor.shape)

(19818, 70)


In [75]:
df_labor = df_all_labor[[
    '地域ＩＤ',
    '男女別コード',
    '15歳以上人口（人） ',
    '15歳以上人口／労働力人口（人）',
    '15歳以上人口／非労働力人口（人）',
    '15歳以上人口／非労働力人口／家事（人）',       # 専業主婦
    '15歳以上人口／非労働力人口／通学者（人）',     # 高校生・大学生
    '15歳以上人口／非労働力人口／その他（人）',     # 幼児・高齢者
    '(再掲)15歳以上人口／労働力人口／就業者／うち雇用者（人）',     # サラリーマン
    '(再掲)15歳以上人口／労働力人口／就業者／うち役員（人）',       # 役員
    '(再掲)15歳以上人口／労働力人口／就業者／うち自営業主（人）',   # 自営業者
    '(再掲)15歳以上人口／労働力人口／就業者／家事のほか仕事（人）',     # パートタイム
    '(再掲)15歳以上人口／労働力人口／就業者／通学のかたわら仕事（人）', # 学生アルバイト
]].copy()
df_labor.columns = [
    'town_id',
    '男女',
    '15歳以上人口（人）',
    '労働力人口（人）',
    '非労働力人口（人）',
    '非労働力_専業主婦（人）',
    '非労働力_学生（人）',
    '非労働力_幼児・高齢者（人）',
    '労働力_サラリーマン（人）',
    '労働力_役員（人）',
    '労働力_自営業者（人）',
    '労働力_パートタイム（人）',
    '労働力_学生アルバイト（人）',
]
df_labor = df_labor.replace('X', 0).replace('-', 0)
df_labor = df_labor[df_labor['男女'] == '1'].reset_index(drop=True)
labor_cols = [col for col in df_labor.columns if '（人）' in col]
for col in labor_cols:
    df_labor[col] = df_labor[col].astype(int)
print(df_labor.shape)
assert df_labor['town_id'].nunique() == len(df_labor)

(6606, 13)


In [76]:
for col in labor_cols:
    if col != '15歳以上人口（人）':
        new_col = col.replace('（人）', '割合%')
        df_labor[new_col] = df_labor[col] / df_labor['15歳以上人口（人）']

df_labor = df_labor.drop(columns=labor_cols+['男女'])
print(df_labor.shape)

(6606, 11)


## 住居の種類

In [77]:
csv_file_6 = 'kd15zv06100000.csv'
df_all_residence = pd.read_csv(os.path.join(temp_dir, csv_file_6), dtype='str')
df_all_residence = df_all_residence[df_all_residence['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_residence.shape)

(6606, 50)


In [78]:
df_residence = df_all_residence[[
    '地域ＩＤ',
    '男女別コード',
    '住宅に住む一般世帯／主世帯／世帯数（世帯）',
    '住宅に住む一般世帯／主世帯／持ち家／世帯数（世帯）',
    '住宅に住む一般世帯／主世帯／公営・都市再生機構・公社の借家／世帯数（世帯）',
    '住宅に住む一般世帯／主世帯／民営の借家／世帯数（世帯）',
    '住宅に住む一般世帯／主世帯／給与住宅／世帯数（世帯）',
]].copy()
df_residence.columns = [
    'town_id',
    '男女',
    '一般世帯_世帯数',
    '持ち家_世帯数',
    '公営借家_世帯数',
    '民営借家_世帯数',
    '社宅_世帯数', 
]
df_residence = df_residence.replace('X', 0).replace('-', 0)
df_residence = df_residence[df_residence['男女'] == '1'].reset_index(drop=True)
residence_cols = [col for col in df_residence.columns if '_世帯数' in col]
for col in residence_cols:
    df_residence[col] = df_residence[col].astype(int)
print(df_residence.shape)
assert df_residence['town_id'].nunique() == len(df_residence)

(6606, 7)


In [79]:
for col in residence_cols:
    if col != '一般世帯_世帯数':
        new_col = col.replace('世帯数', '割合%')
        df_residence[new_col] = df_residence[col] / df_residence['一般世帯_世帯数']

df_residence = df_residence.drop(columns=residence_cols+['男女'])
print(df_residence.shape)

(6606, 5)


## 住宅の建て方

In [80]:
csv_file_7 = 'kd15zv07100000.csv'
df_all_building = pd.read_csv(os.path.join(temp_dir, csv_file_7), dtype='str')
df_all_building = df_all_building[df_all_building['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_building.shape)

(6606, 65)


In [81]:
df_building = df_all_building[[
    '地域ＩＤ',
    '男女別コード',
    '住宅に住む一般世帯／世帯数（世帯）',
    '住宅に住む一般世帯／一戸建／世帯数（世帯）',
    '住宅に住む一般世帯／長屋建／世帯数（世帯）',
    '住宅に住む一般世帯／共同住宅／１・２階建／世帯数（世帯）',
    '住宅に住む一般世帯／共同住宅／３～５階建／世帯数（世帯）',
    '住宅に住む一般世帯／共同住宅／６～10階建／世帯数（世帯）',
    '住宅に住む一般世帯／共同住宅／11階建以上／世帯数（世帯）',
]].copy()
df_building.columns = [
    'town_id',
    '男女',
    '一般世帯_世帯数',
    '一戸建_世帯数',
    '長屋建_世帯数',
    '共同住宅_１・２階建_世帯数',
    '共同住宅_３～５階建_世帯数',
    '共同住宅_６～10階建_世帯数',
    '共同住宅_11階建以上_世帯数',
]
df_building = df_building.replace('X', 0).replace('-', 0)
df_building = df_building[df_building['男女'] == '1'].reset_index(drop=True)
building_cols = [col for col in df_building.columns if '_世帯数' in col]
for col in building_cols:
    df_building[col] = df_building[col].astype(int)
print(df_building.shape)
assert df_building['town_id'].nunique() == len(df_building)

(6606, 9)


In [82]:
for col in building_cols:
    if col != '一般世帯_世帯数':
        new_col = col.replace('世帯数', '割合%')
        df_building[new_col] = df_building[col] / df_building['一般世帯_世帯数']

df_building = df_building.drop(columns=building_cols+['男女'])
print(df_building.shape)

(6606, 7)


## 全てのデータをマージ

In [83]:
df_population_master = pd.merge(df_popdata, df_family, on='town_id', how='left')
df_population_master = pd.merge(df_population_master, df_labor, on='town_id', how='left')
df_population_master = pd.merge(df_population_master, df_residence, on='town_id', how='left')
df_population_master = pd.merge(df_population_master, df_building, on='town_id', how='left')
print(df_population_master.shape)

(3145, 60)


## 出力

In [84]:
df_population_master.to_csv(os.path.join(DIR_NAME, 'population_master.csv'), index=False, encoding='utf_8_sig')

In [85]:
list(df_population_master.columns)

['town_id',
 '市区町村名',
 '町名',
 '町丁目',
 '人口総数（人）',
 '面積（km2）',
 '人口密度(人/km2)',
 '0〜4歳（人）',
 '5〜9歳（人）',
 '10〜14歳（人）',
 '15〜19歳（人）',
 '20〜24歳（人）',
 '25〜29歳（人）',
 '30〜34歳（人）',
 '35〜39歳（人）',
 '40〜44歳（人）',
 '45〜49歳（人）',
 '50〜54歳（人）',
 '55〜59歳（人）',
 '60〜64歳（人）',
 '65〜69歳（人）',
 '70〜74歳（人）',
 '75〜79歳（人）',
 '80〜84歳（人）',
 '85〜89歳（人）',
 '90〜94歳（人）',
 '95〜99歳（人）',
 '100歳以上（人）',
 '年齢不詳（人）',
 '平均年齢（歳）',
 '外国人（人）',
 '対象',
 '世帯総数',
 '1人世帯',
 '2人世帯',
 '3人世帯',
 '4人世帯',
 '5人以上世帯',
 '世帯人員（人）',
 '1世帯当たり人員（人）',
 '労働力人口割合%',
 '非労働力人口割合%',
 '非労働力_専業主婦割合%',
 '非労働力_学生割合%',
 '非労働力_幼児・高齢者割合%',
 '労働力_サラリーマン割合%',
 '労働力_役員割合%',
 '労働力_自営業者割合%',
 '労働力_パートタイム割合%',
 '労働力_学生アルバイト割合%',
 '持ち家_割合%',
 '公営借家_割合%',
 '民営借家_割合%',
 '社宅_割合%',
 '一戸建_割合%',
 '長屋建_割合%',
 '共同住宅_１・２階建_割合%',
 '共同住宅_３～５階建_割合%',
 '共同住宅_６～10階建_割合%',
 '共同住宅_11階建以上_割合%']

### 高級タワマン指数

In [86]:
score_family = df_population_master['世帯総数'].quantile(0.1)
score_housewife = df_population_master['非労働力_専業主婦割合%'].quantile(0.8)
score_towerapartment = df_population_master['共同住宅_11階建以上_割合%'].quantile(0.8)
score_ownedhouse = df_population_master['持ち家_割合%'].quantile(0.8)
score_boardmember = df_population_master['労働力_役員割合%'].quantile(0.9)
df_copy = df_population_master.copy()
df_copy['高級タワマン指数'] = df_copy['非労働力_専業主婦割合%'] * df_copy['共同住宅_11階建以上_割合%'] * df_copy['持ち家_割合%'] * df_copy['労働力_役員割合%']*1000
df_copy[
    (df_copy['対象'] == 1) &
    (df_copy['世帯総数'] > score_family) &
    (df_copy['非労働力_専業主婦割合%'] > score_housewife) &
    (df_copy['共同住宅_11階建以上_割合%'] > score_towerapartment) &
    (df_copy['持ち家_割合%'] > score_ownedhouse) &
    (df_copy['労働力_役員割合%'] > score_boardmember)
].sort_values('高級タワマン指数', ascending=False)[[
    'town_id',
    '市区町村名',
    '町丁目',
    '世帯総数',
    '人口総数（人）',
    '面積（km2）',
    '人口密度(人/km2)',
    '高級タワマン指数',
    '非労働力_専業主婦割合%',
    '共同住宅_11階建以上_割合%',
    '持ち家_割合%',
    '労働力_役員割合%',
]]

,town_id,市区町村名,町丁目,世帯総数,人口総数（人）,面積（km2）,人口密度(人/km2),高級タワマン指数,非労働力_専業主婦割合%,共同住宅_11階建以上_割合%,持ち家_割合%,労働力_役員割合%
1642,113011,渋谷区,広尾４丁目,1941,4599,0.25,18396.0,5.940879,0.131880,0.585827,0.794521,0.096783
382,104060,新宿区,神楽坂５丁目,379,703,0.02,35150.0,5.825249,0.127243,0.685185,0.744681,0.089723
206,102115,中央区,勝どき６丁目,3073,6995,0.10,69950.0,5.396101,0.128023,0.918893,0.615686,0.074502
1509,112190,世田谷区,玉川１丁目,1609,3443,0.30,11476.7,3.981270,0.143224,0.569116,0.751412,0.065002
1709,113095,渋谷区,神宮前１丁目,1026,1933,0.22,8786.4,2.304528,0.132353,0.356842,0.613419,0.079545
315,103128,港区,白金台１丁目,499,1080,0.14,7714.3,2.302064,0.126087,0.341365,0.615071,0.086957
317,103130,港区,白金台３丁目,1742,3668,0.16,22925.0,2.186201,0.137375,0.341001,0.692441,0.067398


## 高級住宅指数

In [87]:
score_family = df_population_master['世帯総数'].quantile(0.1)
score_housewife = df_population_master['非労働力_専業主婦割合%'].quantile(0.8)
score_towerapartment = df_population_master['一戸建_割合%'].quantile(0.8)
score_ownedhouse = df_population_master['持ち家_割合%'].quantile(0.8)
score_boardmember = df_population_master['労働力_役員割合%'].quantile(0.9)
df_copy = df_population_master.copy()
df_copy['高級住宅指数'] = df_copy['非労働力_専業主婦割合%'] * df_copy['一戸建_割合%'] * df_copy['持ち家_割合%'] * df_copy['労働力_役員割合%']*1000
df_copy[
    (df_copy['対象'] == 1) &
    (df_copy['世帯総数'] > score_family) &
    (df_copy['非労働力_専業主婦割合%'] > score_housewife) &
    (df_copy['一戸建_割合%'] > score_towerapartment) &
    (df_copy['持ち家_割合%'] > score_ownedhouse) &
    (df_copy['労働力_役員割合%'] > score_boardmember)
].sort_values('高級住宅指数', ascending=False)[[
    'town_id',
    '市区町村名',
    '町丁目',
    '世帯総数',
    '人口総数（人）',
    '面積（km2）',
    '人口密度(人/km2)',
    '高級住宅指数',
    '非労働力_専業主婦割合%',
    '一戸建_割合%',
    '持ち家_割合%',
    '労働力_役員割合%',
]].head(10)

,town_id,市区町村名,町丁目,世帯総数,人口総数（人）,面積（km2）,人口密度(人/km2),高級住宅指数,非労働力_専業主婦割合%,一戸建_割合%,持ち家_割合%,労働力_役員割合%
1234,111117,大田区,田園調布３丁目,699,1690,0.33,5121.2,25.897738,0.193640,0.938865,0.861401,0.165371
1235,111118,大田区,田園調布４丁目,832,1919,0.34,5644.1,11.393854,0.186298,0.770631,0.758962,0.104567
1538,112225,世田谷区,成城５丁目,851,2048,0.21,9752.4,8.359911,0.166667,0.760282,0.761051,0.086689
1251,111138,大田区,久が原６丁目,868,2085,0.16,13031.3,7.796514,0.176022,0.682243,0.696682,0.093188
1236,111119,大田区,田園調布５丁目,1373,3297,0.42,7850.0,6.817677,0.175254,0.652747,0.702602,0.084823
1484,112159,世田谷区,尾山台１丁目,721,1645,0.14,11750.0,5.955208,0.154736,0.648725,0.713459,0.083153
1537,112224,世田谷区,成城４丁目,1376,3361,0.32,10503.1,5.445417,0.179015,0.561047,0.831245,0.065225
1249,111136,大田区,久が原４丁目,1617,4017,0.28,14346.4,5.336988,0.164487,0.660236,0.655435,0.074978
1539,112226,世田谷区,成城６丁目,694,1561,0.29,5382.8,5.230809,0.141727,0.575624,0.670103,0.095683
1485,112160,世田谷区,尾山台２丁目,881,1936,0.18,10755.6,5.035552,0.161998,0.608400,0.687643,0.074300
